# Exploring CESM output

## full, high res output data:
- grid data is not stored, nlat/nlon are natural numbers, depths stored in centimeter
- grid data is stored in binary files

## interpolated, rect output data
- grid: u/t_lon/lat store position in degrees, depth_t/w_dep store deth in meters

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
# plt.style.use('dark_background')
sns.set()
sns.set_palette("tab10")
sns.set_context("notebook", font_scale=1)

In [ ]:
from paths import path_CESM
from paths import path_ocn_ctrl, path_ocn_ctrl_rect
from paths import path_ocn_rcp,  path_ocn_rcp_rect

In [ ]:
nc_file = f'{path_ocn_rcp}/rcp8.5_co2_f05_t12.pop.h.2000-01.nc'
xdata = xr.open_dataset(nc_file, decode_times=False)

In [ ]:
xdata.coords['t_lon']

In [ ]:
xdata['TEMP']

In [ ]:
xdata['TEMP'][0,:,:]

## map of daily file

In [ ]:
nc_file = f'{path_CESM}/rcp8.5_co2_f05_t12/OUTPUT/ocn/hist/daily/rcp8.5_co2_f05_t12.pop.hm.2021-01-22.nc'
xdata = xr.open_dataset(nc_file, decode_times=False)

In [ ]:
from read_binary import read_binary_2D

In [ ]:
from grid import generate_lats_lons, shift_field

In [ ]:
grid_file = f'{path_CESM}/inputdata/ocn/pop/tx0.1v2/grid/horiz_grid_200709.ieeer8'

imt,jmt,km        = 3600, 2400, 42
# grid_file       = '../../input/grid.3600x2400.fob.da'
lats,lons,shift   = generate_lats_lons(grid_file)

In [ ]:
f, ax = plt.subplots(1,3,figsize=(15,5), sharey=True)
ax[0].imshow(TEMP.T, cmap='viridis')
ax[1].imshow(lats.T)
ax[2].imshow(lons.T)
plt.tight_layout()

In [ ]:
def map_geography(ax):
    """adds land, WGKP ploygon, lat/lon labels"""
    xticks = np.linspace(-180,180,7,endpoint=True)
    yticks = np.linspace(-90,90,7)
    ax.add_feature(cartopy.feature.LAND, zorder=1, facecolor='lightgray')
    gl = ax.gridlines(xlocs=xticks, ylocs=yticks)
    gl.n_steps = 500

In [ ]:
TEMP = shift_field(xdata.TEMP[0,0,:,:].T,shift)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax  = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(central_latitude=-90))
ax.set_position([.02,.02,.96,.96])
ax.pcolormesh(lons.T, lats.T, TEMP.T,
              cmap='rainbow',
              vmin=-1.8, vmax=27,
              transform=ccrs.PlateCarree() )
map_geography(ax)
fig.savefig('../../results/first_map', transparent=True, dpi=300)

In [ ]:
xdata

In [ ]:
def sample_data(shape=(73, 145)):
    """Return ``lons``, ``lats`` and ``data`` of some fake data."""
    nlats, nlons = shape
    lats = np.linspace(-np.pi / 2, np.pi / 2, nlats)
    lons = np.linspace(0, 2 * np.pi, nlons)
    lons, lats = np.meshgrid(lons, lats)
    wave = 0.75 * (np.sin(2 * lats) ** 8) * np.cos(4 * lons)
    mean = 0.5 * np.cos(2 * lats) * ((np.sin(2 * lats)) ** 2 + 2)

    lats = np.rad2deg(lats)
    lons = np.rad2deg(lons)
    data = wave + mean

    return lons, lats, data

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

lons, lats, data = sample_data()

ax.contourf(lons, lats, data,
            transform=ccrs.PlateCarree(),
            cmap='nipy_spectral')
ax.coastlines()
ax.set_global()
plt.show()